In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import csv
import sys
import numpy as np
import cugraph as cg
import cudf, cuml
import pandas as pd
import cupy as cp

In [2]:
labels=['Art','Collectible','Games','Metaverse','Utility','Other']
path="./data/Data_API_mapped.csv"

In [8]:
df=cudf.read_csv(path)
df=df.iloc[:,0:2]
df.columns = ["Seller","Buyer"]
print(df.head())

   Seller  Buyer
0       0      2
1       0      1
2       0      2
3       0      2
4       0      2


In [9]:
G = cg.MultiGraph()
G.from_cudf_edgelist(df, source='Seller', destination='Buyer')
#Ouput the number of nodes and edges
nodes = G.number_of_nodes()
edges = G.number_of_edges()
print("Number of nodes and edges: ",nodes,"  ",edges)

Number of nodes and edges:  532945    6073872


In [ ]:
sum_of_in_degree=0
in_degree_list=[]
sum_of_out_degree=0
out_degree_list=[]
degree_list=[]
for i,node in enumerate(G.nodes().to_numpy()):
    temp_in=G.in_degree(node).to_numpy()[0][1]
    temp_out=G.out_degree(node).to_numpy()[0][1]
    sum_of_in_degree+=temp_in
    sum_of_out_degree+=temp_out

    degree_list.append(temp_in+temp_out)
    if i%1000==0:
        print('----{} nodes----'.format(i))


In [ ]:
avg_in_degree=sum_of_in_degree/nodes
avg_out_degree=sum_of_out_degree/nodes
print("2. Average in-degree: ",avg_in_degree)
print("3. Average out-degree: ",avg_out_degree)
#3. Ouput the average degree(i.e., sum of in-degree and out-degree)
avg_degree=(sum_of_in_degree+sum_of_out_degree)/nodes
print("5. Average degree: ",avg_degree)

In [ ]:
from collections import Counter
degrees, degree_freq = np.unique(degree_list, return_counts=True)
#log-log
plt.title("Degree Histograms ")
plt.loglog(degrees.get(),degree_freq.get(),'ro-')
plt.xlabel('Degree,k')
plt.ylabel('Count,P(K)*n')

In [ ]:
# compute distance
dis_seq = []
for i, node in enumerate(G.nodes().to_numpy()):
    path_len = cg.sssp(G, source=node)
    dis_seq.extend(path_len['distance'].to_numpy())
    if i % 1000 == 0:
        print("-----{} node----".format(i))
dis, dis_freq = np.unique(dis_seq, return_counts=True)
dis=dis[:-1]
dis_freq=dis_freq[:-1]
#log-log
plt.title("Path Distance")
plt.loglog(dis, dis_freq, 'ro-')
plt.xlabel('Distance (hops)')
plt.ylabel('Number of paths')
#Calculate average path
dis_seq=np.array(dis_seq)
dis_seq=dis_seq[dis_seq<10**10]
avg_path = sum(dis_seq) / (len(dis_seq) - nodes)
print(avg_path)